In [1]:
import ee
import geemap
import os
import datetime as dt
import numpy as np

In [3]:
!pip install ffmpeg-python

In [4]:
ee.Authenticate()

Enter verification code:  4/1ATx3LY6qA7wRCh4QxrC1cf3lJsmHlL6vnSQzE3Rf8i4YOoMAEloMOrce5Xg



Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
region = ee.Geometry.BBox(-51, -78.5, -41, -76.5) # Landfast ice in the Weddell Sea

# # Mukund's ROI
# bbox = [-10, 72.6, 12, 79.2]
# # bbox = [-176.3, -76.3, -153.9, -74.4]
# region = ee.Geometry.BBox(bbox[0], bbox[1], bbox[2], bbox[3]) #ee.Geometry.Rectangle(bbox)
# point = ee.Geometry.Point([np.mean([bbox[0], bbox[2]]), np.mean([bbox[1], bbox[3]])]).buffer(20000)
# # Approximately (72.6 to 79.2 N) and (-10 to 12 E) April to May 2023
# # Approximately (-76.3 to -74.4 S) and (-176.3 to -153.9 W) November - December 2023

# Grounded icebergs ROI
# region = ee.Geometry.Point([-45.14359469618727,-76.48507480890505]).buffer(50000)

# region = ee.Geometry.Polygon([
#     [-51, -78.5], [-51, -76.5], [-41, -76.5], [-41, -78.5], [-51, -78.5]
# ])

# point = ee.Geometry.Point([-46, -77.5])

collection = ee.ImageCollection("COPERNICUS/S1_GRD")\
.filter(ee.Filter.eq('transmitterReceiverPolarisation', ["HH"]))\
.filterBounds(region)
# .filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_start', 5), ee.Filter.eq('relativeOrbitNumber_start', 34),
#         ee.Filter.eq('relativeOrbitNumber_start', 136)))

# 2016: 165
# 2017: 5

### Generate GIF time series

In [4]:
def collection_addbands(img):
    bands = img.bandNames() # First band ('HH' or 'VV')
    band = [ee.Algorithms.If(bands.contains('HH'), 'HH', 'VV')]
    norm = img.select(band).divide(img.select('angle')).rename('norm')
    img2 = img.addBands(norm, overwrite=True).select('norm')
    return img2

def add_coverage(img):
    
    tol = 10000
    overlap = img.geometry().intersection(region, tol)
    ratio = overlap.area(tol).divide(region.area(tol))
    return img.set({'coverage_ratio': ratio})

# calculate coverage area of image to roi
def coverage(img):
    tol = 10000
    overlap = img.geometry().intersection(region, tol)
    ratio = overlap.area(tol).divide(region.area(tol))
    return ratio.getInfo()

In [9]:
for year in np.arange(2015, 2018):
    step = 3
    for month in range(1, 13, step):
        start_date = f"{year}-{str(month).zfill(2)}-01"
        if month < 12-step:
            end_date = f"{year}-{str(month+step).zfill(2)}-01"
        else:
            end_date = f"{year+1}-01-01"
        
        collection0 = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(region).filterDate(start_date, end_date)
        collection1 = collection0.map(collection_addbands)
        collection = collection1.map(add_coverage).filter(ee.Filter.gt('coverage_ratio', 0.5))

        video_args = {
            "dimensions": 600,
            "region": region,
            "framesPerSecond": 1.5,
            "bands": "norm",
            "crs": "EPSG:3857", #
            "min": -0.8,
            "max": -0.2,
            "gamma": 1    
        }
        
        images = geemap.create_timeseries(
            collection, start_date, end_date, region, frequency="day", reducer="median"
        )
        
        s = images.aggregate_array('system:time_start').getInfo()
        
        days = []
        for sec in s:
            d = dt.datetime(1970, 1, 1) + dt.timedelta(seconds = sec / 1000)
            days.append(d.strftime("%Y-%m-%d"))
        
        print(start_date, end_date, images.size().getInfo())
        
        work_dir = "D:\\Floes\\S1"
        
        out_gif = os.path.join(work_dir, f"S1_landfast_{start_date}_{end_date}.gif")
        geemap.download_ee_video(images, video_args, out_gif)
        
        texted_gif = out_gif.replace(".gif", "_text.gif") #os.path.join(work_dir, f"S1_{hs}_{start_date}_{end_date}_text.gif")
        geemap.add_text_to_gif(
            out_gif,
            texted_gif,
            xy=("3%", "3%"),
            text_sequence=days,
            font_size=24,
            duration=800,
            font_color="#ff0000",
            add_progress_bar=False,
        )

        if os.path.exists(out_gif):
            os.remove(out_gif)
        
        print("====================================================")

print("DONE!")

2015-01-01 2015-04-01 46
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_landfast_2015-01-01_2015-04-01.gif
2015-04-01 2015-07-01 35
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_landfast_2015-04-01_2015-07-01.gif
2015-07-01 2015-10-01 11
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_landfast_2015-07-01_2015-10-01.gif
2015-10-01 2016-01-01 4
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_landfast_2015-10-01_2016-01-01.gif
2016-01-01 2016-04-01 9
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_landfast_2016-01-01_2016-04-01.gif
2016-04-01 2016-07-01 12
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_landfast_2016-04-01_2016-07-01.gif
2016-07-01 2016-10-01 11
Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\S1\S1_landfast_2016-07-01_2016-10-01.gif
2016-10-

In [22]:
import imageio.v3 as iio
import numpy as np
import glob

gif_files = glob.glob("D:\\Floes\\S1\\S1_landfast_20*.gif")
combined = [iio.imread(f) for f in gif_files]

frames = np.vstack(combined)

# get duration each frame is displayed
duration = iio.immeta(gif_files[0])["duration"]/2

iio.imwrite("D:\\Floes\\S1\\S1_landfast_combined.gif", frames, duration=duration)

In [18]:
import ffmpy

ff = ffmpy.FFmpeg(inputs={"D:\\Floes\\S1\\S1_landfast_combined.gif": None},
                  outputs={"D:\\Floes\\S1\\S1_landfast_video.mp4": None})
ff.run()

(None, None)

In [ ]:
import moviepy.editor as mp

clip = mp.VideoFileClip("D:\\Floes\\S1\\S1_landfast_combined.gif")
clip.write_videofile("D:\\Floes\\S1\\S1_landfast_video.mp4")

In [7]:
start_date = "2015-04-01"
end_date = "2017-07-01"
region = ee.Geometry.BBox(-51, -78.5, -41, -76.5)

In [8]:
images = geemap.create_timeseries(
    collection, start_date, end_date, region, frequency="day", reducer="median"
)

s = images.aggregate_array('system:time_start').getInfo()
days = []
for sec in s:
    d = dt.datetime(1970, 1, 1) + dt.timedelta(seconds = sec / 1000)
    days.append(d.strftime("%Y-%m-%d"))

print(images.size().getInfo())

63


In [9]:

# Define arguments for animation function parameters.
video_args = {
    "dimensions": 600,
    "region": region,
    "framesPerSecond": 2,
    "bands": ["HH"],
    "crs":'EPSG:3857',
    "min": -25,
    "max": -5,
    "gamma": 1
    
}

In [10]:
work_dir = "D:\\Floes\\figures" #os.path.join(os.path.expanduser("~"), "Downloads")
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
out_gif = os.path.join(work_dir, f"sentienl1_{start_date}_{end_date}.gif")

In [11]:
geemap.download_ee_video(images, video_args, out_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: D:\Floes\figures\sentienl1_2017-04-01_2017-07-01.gif


In [12]:
texted_gif = os.path.join(work_dir, f"sentienl1_{start_date}_{end_date}_text.gif")
geemap.add_text_to_gif(
    out_gif,
    texted_gif,
    xy=("3%", "5%"),
    text_sequence=days,
    font_size=20,
    duration=800,
    font_color="#ffffff",
    add_progress_bar=False,
)

geemap.show_image(texted_gif)

Output()

Image(value=b'GIF89aX\x02,\x02\x87\x1f\x00\xff\xff\xff\xfe\xfe\xfe\xfd\xfd\xfd\xfc\xfc\xfc\xfb\xfb\xfb\xfa\xfa…

In [30]:
timelapse = geemap.sentinel1_timelapse(
    region,
    out_gif="sentinel1.gif",
    start_year=2018,
    end_year=2019,
    start_date="01-01",
    end_date="03-01",
    dimensions=1000,
    frequency="week",
    bands=['HH'],
    reducer='first',
    vis_params={"min": -25, "max": -5},
    palette="Greys",
    frames_per_second=1.5,
    title="Sentinel-1 Timelapse",
    add_colorbar=True,
    colorbar_bg_color="gray",
)

print("DONE!")

Total number of images: 61

ffmpeg is not installed on your computer.


In [31]:
geemap.show_image(timelapse)

Output()